In [1]:
import itertools

import numpy
from sympy import init_session

In [2]:
init_session()

IPython console for SymPy 1.0 (Python 2.7.12-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at http://docs.sympy.org/1.0/


In [3]:
C = symbols("c0:4")
S = symbols("s0:4")
A = symbols("a0:4")
B = symbols("b0:4")

In [4]:
def dot(es1, es2):
    return sum([e[0]*e[1] for e in zip(es1, es2)])

In [19]:
def do_integral(f, s):
    return Integral(f, (s, 0, 1))
        # The bounds of 0 and 1 are specific to the Fourier FA. Make
        # sure you normalize the inputs.

In [22]:
da_integral = reduce(do_integral, S[1:], cos(pi * dot(S, C))).doit()

In [36]:
# Copied from hiora_cartpole.fourier_fa.
order = 3
n_dims = 4
c_matrix = np.array(
               list( itertools.product(range(order+1), repeat=n_dims) ),
               dtype=np.int32)

In [37]:
def sum_term(integral, c, c_vec):
    return integral.subs(zip(c, c_vec))

In [38]:
sum_terms = [sum_term(da_integral, C, c_vec) for c_vec in c_matrix]

In [39]:
np_sum_terms = [lambdify(S[0], sum_term, 'numpy') for sum_term in sum_terms]

In [45]:
def phi(npsts, theta, s0):
    ns0 = (s0 - -2.5) / 5.0
    return np.dot(theta, 
                  np.array([npst(ns0) for npst in npsts]))

In [46]:
theta = np.load("theta.npy")

In [47]:
res = np.array([phi(np_sum_terms, theta[512:768], x) 
 for x in np.arange(-2.38, 2.5, 0.5*1.19)])


In [48]:
res

array([  -129838.63097278,   -298998.59748568,   3087412.84846354,
         8130942.03887799,  10758471.51583669,   8664232.14626823,
         3636646.02919975,   -222434.83286789,   -491482.63696056])

In [32]:
other = np.array([-3748598.374407076,
 -8333255.9176837215,
 92906846.75614552,
 242969379.49722022,
 320543060.70953935,
 257463642.38676718,
 107526913.72252564,
 -7061727.3744605975,
 -14631018.954087665])

In [44]:
res/other

array([ 0.03463658,  0.03588017,  0.03323127,  0.03346488,  0.03356326,
        0.03365225,  0.0338208 ,  0.03149864,  0.03359183])

In [34]:
other/res

array([ 28.87120995,  27.87055186,  30.09213582,  29.88207004,
        29.79447966,  29.71569067,  29.56760511,  31.74739893,  29.76914718])

In [66]:
res2 = np.array([phi(np_sum_terms, theta[768:1024], x) 
 for x in np.arange(-2.38, 2.5, 0.5*1.19)])


In [67]:
res2

array([  -174316.21632421,   -505178.62138594,   2743738.37912048,
         7828381.28926773,  10698118.73072955,   8891660.84160946,
         3996424.61865612,     49410.50215361,   -409740.34118969])

In [68]:
res2*29.5

array([ -5.14232838e+06,  -1.49027693e+07,   8.09402822e+07,
         2.30937248e+08,   3.15594503e+08,   2.62303995e+08,
         1.17894526e+08,   1.45760981e+06,  -1.20873401e+07])

This is strange. Why are the results I get with numeric integration approximately 30 times those of the analytic integration?